In [ ]:
import os
from random import randrange,randint
from copy import deepcopy
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import itertools
import uuid

import logging
logging.basicConfig(
    level="WARNING",
    format="%(levelname)s:%(message)s"
)
logger = logging.getLogger(__name__)

from scipy.spatial import Delaunay
from scipy.spatial.distance import cdist

from matplotlib.colors import ListedColormap
plt.rcParams["figure.figsize"] = (2,2)

## Class

In [ ]:
from typing import Mapping, Tuple
from AStarMaze import AStarMaze
import random

class Genotype:
    """
        Types of POIs:
            + `-1`: Blocked
            + `0`,`1`: <Reserved>
            + `2`: Entry  (neutral room)
            + `3`: Exit   (neutral room)
            + `4`: Neutral
            + `5`: Monster
            + `6`: Treasure
        
    """
    def __init__(self):
        self.pois = None
        self.number_of_pois = 5
        self.dungeon_shape = (16, 16)
        self.growth_iterator_limit = 0
    
    def is_valid(self, xy_bounds:Tuple[int,int]) -> bool:
        # test if: (a) the pois locations are within xy bounds, (b) there is one and only one entry and exit, (c) there are no reserved values such as `0`,`1`
        return True
    
    def generate_genotype(self, dungeon_shape:Tuple[int,int], number_of_pois, growth_iterator_limit) -> None:
        if number_of_pois < 3:
            raise ValueError(f"There must be at least 3 number_of_pois, actual={number_of_pois}")
        pois = [[randrange(dungeon_shape[0]), randrange(dungeon_shape[1]), 2],[randrange(dungeon_shape[0]), randrange(dungeon_shape[1]), 3]]
        additional_pois = [[randrange(dungeon_shape[0]), randrange(dungeon_shape[1]), randint(4, 6)] for n in range(number_of_pois-2)]
        self.pois=np.asarray(pois+additional_pois, dtype=int)
        self.number_of_pois = number_of_pois
        self.dungeon_shape = dungeon_shape
        self.growth_iterator_limit = growth_iterator_limit

    def mutate_from_genotype(self, genotomut, global_mutation_strength:float = 1.0):
        def random_move_within_manhattan_np(x, y, x_max, y_max, max_distance):
            """
            Efficiently computes a random move from (x, y) within a specified Manhattan distance
            on a bounded grid using NumPy.

            Args:
                x (int): Current x-coordinate.
                y (int): Current y-coordinate.
                x_max (int): Maximum x bound (inclusive).
                y_max (int): Maximum y bound (inclusive).
                max_distance (int): Maximum Manhattan distance for movement.

            Returns:
                tuple: Randomly selected valid new (x, y) coordinates.
            """

            # Create a grid of offsets within the max_distance
            dx = np.arange(-max_distance, max_distance + 1)
            dy = np.arange(-max_distance, max_distance + 1)
            dx_grid, dy_grid = np.meshgrid(dx, dy)

            # Compute Manhattan distance for all offset pairs
            manhattan_dist = np.abs(dx_grid) + np.abs(dy_grid)
            mask = (manhattan_dist <= max_distance) & (manhattan_dist > 0)

            # Apply mask to get valid offsets
            dx_valid = dx_grid[mask]
            dy_valid = dy_grid[mask]

            # Compute all candidate new positions
            new_x = x + dx_valid
            new_y = y + dy_valid

            # Bound checking
            within_bounds = (0 <= new_x) & (new_x < x_max) & (0 <= new_y) & (new_y < y_max)
            new_x = new_x[within_bounds]
            new_y = new_y[within_bounds]

            if len(new_x) == 0:
                return (x, y)  # No valid moves

            idx = np.random.randint(len(new_x))
            return int(new_x[idx]), int(new_y[idx])

        # Mutate poi
        dungeon_min_dimension = min(genotomut.dungeon_shape[0],genotomut.dungeon_shape[1]) 
        mutation_strength_poi_move = round(global_mutation_strength * (dungeon_min_dimension / 4)) # 2 is a bit strong
        mutation_strength_poi_change_chance = 0.5 * global_mutation_strength
        pois_existing_mutated = []
        for poi in genotomut.pois:
            x, y = random_move_within_manhattan_np(x=poi[0], y=poi[1], x_max=genotomut.dungeon_shape[0], y_max=genotomut.dungeon_shape[1], max_distance=mutation_strength_poi_move)
            if poi[2] > 3:
                if random.random() + mutation_strength_poi_change_chance > 1:
                    pois_existing_mutated.append([x, y, random.choice([4,5,6])])
                else:
                    pois_existing_mutated.append([x, y, poi[2]])
            else:
                pois_existing_mutated.append([x, y, poi[2]])
        logger.debug(pois_existing_mutated)

        # Mutate poi count
        mutated_pois = []
        mutation_strength_poi_number_dx = 1.5 * global_mutation_strength
        mutated_number_of_pois = round(random.uniform(max(3, genotomut.number_of_pois - mutation_strength_poi_number_dx), genotomut.number_of_pois + mutation_strength_poi_number_dx))

        if mutated_number_of_pois > genotomut.number_of_pois:
            pois_difference = abs(mutated_number_of_pois-genotomut.number_of_pois)
            # Add new random pois
            new_pois = [[randrange(genotomut.dungeon_shape[0]), randrange(genotomut.dungeon_shape[1]), randint(4, 6)] for n in range(pois_difference)]
            mutated_pois = pois_existing_mutated + new_pois
        elif mutated_number_of_pois < genotomut.number_of_pois:
            pois_difference = abs(mutated_number_of_pois-genotomut.number_of_pois)
            arr_pois = np.asarray(genotomut.pois)
            potential_pois = arr_pois[(arr_pois[:, 2] != 2) & (arr_pois[:, 2] != 3)].tolist()
            endpoints_pois = arr_pois[(arr_pois[:, 2] == 2) | (arr_pois[:, 2] == 3)].tolist()
            mutated_pois = endpoints_pois + random.sample(potential_pois, k=pois_difference)
        else:
            mutated_pois = pois_existing_mutated

        # Mutate grow range
        mutation_strength_growth_dx = 1.5 * global_mutation_strength
        mutated_growth_iterator_limit = round(random.uniform(max(1, genotomut.growth_iterator_limit - mutation_strength_growth_dx), genotomut.growth_iterator_limit + mutation_strength_growth_dx))

        def __repr__(self) -> str:
            return str(f"pois={self.pois}\n shape={self.dungeon_shape}\n growth_limit={self.growth_iterator_limit}")
        
        ## ASSIGN
        self.pois = np.asarray(mutated_pois)
        self.number_of_pois = mutated_number_of_pois
        self.dungeon_shape = self.dungeon_shape
        self.growth_iterator_limit = mutated_growth_iterator_limit

    def __repr__(self) -> str:
        return str(f"pois={self.pois}, nbpois={self.number_of_pois}, growth={self.growth_iterator_limit}")

# class AStarMaze():
#     class AStarNode():
#         """A node class for A* Pathfinding"""

#         def __init__(self, parent=None, position=None):
#             self.parent = parent
#             self.position = position

#             self.g = 0
#             self.h = 0
#             self.f = 0

#         def __eq__(self, other):
#             return self.position == other.position
        
#     def __init__(self, maze) -> None:
#         self.maze=maze

#     def solve_shortest_path(self, start, end):
#         logger.info(f"--- Start solving Maze' Shortest Path ---")
#         """Returns a list of tuples as a path from the given start to the given end in the given maze"""

#         # Create start and end node
#         start_node = self.AStarNode(None, start)
#         start_node.g = start_node.h = start_node.f = 0
#         end_node = self.AStarNode(None, end)
#         end_node.g = end_node.h = end_node.f = 0

#         # Initialize both open and closed list
#         open_list = []
#         closed_list = []

#         # Add the start node
#         open_list.append(start_node)

#         # Loop until you find the end
#         while len(open_list) > 0:

#             # Get the current node
#             current_node = open_list[0]
#             current_index = 0
#             for index, item in enumerate(open_list):
#                 if item.f < current_node.f:
#                     current_node = item
#                     current_index = index

#             # Pop current off open list, add to closed list
#             open_list.pop(current_index)
#             closed_list.append(current_node)

#             # Found the goal
#             if current_node == end_node:
#                 path = []
#                 current = current_node
#                 while current is not None:
#                     path.append(current.position)
#                     current = current.parent
#                 return path[::-1] # Return reversed path

#             # Generate children
#             children = []
#             for new_position in [(0, -1), (0, 1), (-1, 0), (1, 0), (-1, -1), (-1, 1), (1, -1), (1, 1)]: # Adjacent squares

#                 # Get node position
#                 node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])

#                 # Make sure within range
#                 if node_position[0] > (len(self.maze) - 1) or node_position[0] < 0 or node_position[1] > (len(self.maze[len(self.maze)-1]) -1) or node_position[1] < 0:
#                     continue

#                 # Make sure walkable terrain
#                 if self.maze[node_position[0]][node_position[1]] != 0:
#                     continue

#                 # Create new node
#                 new_node = self.AStarNode(current_node, node_position)

#                 # Append
#                 children.append(new_node)

#             # Loop through children
#             for child in children:

#                 # Child is on the closed list
#                 for closed_child in closed_list:
#                     if child == closed_child:
#                         continue

#                 # Create the f, g, and h values
#                 child.g = current_node.g + 1
#                 child.h = ((child.position[0] - end_node.position[0]) ** 2) + ((child.position[1] - end_node.position[1]) ** 2)
#                 child.f = child.g + child.h

#                 # Child is already in the open list
#                 for open_node in open_list:
#                     if child == open_node and child.g > open_node.g:
#                         continue

#                 # Add the child to the open list
#                 open_list.append(child)


class DungeonMap:
    def __init__(self, dimension:int, genotype:Genotype):
        # self._id = str(uuid.uuid4())[:8]
        self.dungeon_shape= (dimension,dimension)
        self.set_dungeon_Genotype(genotype=genotype)
        self.dungeonmap=None
        self.shortest_path=None
        self.shortest_path_rooms=None
        self.feature_dict={}
        self.genotype = genotype

    # Genotype
    def set_dungeon_Genotype(self, genotype: Genotype):
        if genotype is not None:
            if genotype.is_valid(xy_bounds=self.dungeon_shape):        
                self.genotype=genotype
            else:
                raise ValueError(f"Provided Genotype is not valid regarding DungeonMap spec, maybe some pois out of bounds ?")

    def get_dungeon_entrypoints_positions(self, upscaled):
        d_in = self.genotype.pois[np.where(self.genotype.pois[:,2] == 2)][0,:2]
        d_out = self.genotype.pois[np.where(self.genotype.pois[:,2] == 3)][0,:2]
        if upscaled is True:
            return tuple(d_in*2), tuple(d_out*2)
        else:
            return tuple(d_in), tuple(d_out)

    # Dungeon features extraction
    def extract_dungeon_features(self) -> dict | None:
        """
            + corridorness(int): the corridor/room tile ratio, for the whole dungeon
            + nolliness(int): the ratio of empty to walkable spaces
            + spath_length(int): the length of the shortest path
            + spath_rooms_types_completeness_ratio(int): the ratio of the types of rooms crossed by the shortest path, on the number of possible rooms let by the Genotype. If =1, all the Genotype' room types are crossed
            + spath_rooms_diversity_ratio(int): measures the variety of rooms crossed (not always the same)
            + spath_corridorness(int): the corridor/room tile ratio, for the shortest path

        Raises:
            ValueError: _description_

        Returns:
            bool: _description_
        """

        ## Dungeon map
        feature_dict = {}
        if not isinstance(self.dungeonmap, np.ndarray):
            raise ValueError(f"Compute the dungeon map before extracting any featyre, dungeonmap={self.dungeonmap}")
        
        ### Corrdiorness
        tiles_count=(self.dungeonmap.shape[0] * self.dungeonmap.shape[1])
        corridors_count = np.count_nonzero(self.dungeonmap == 1)
        wall_count = np.count_nonzero(self.dungeonmap == 0)
        feature_dict["corridorness"] = corridors_count / (tiles_count - wall_count)
    
        ### Nolliness,the ratio of empty to walkable spaces. 1=full walkable
        arr_walkable = self._get_walkable_dungeon_surfaces()
        feature_dict["nolliness"] = 1 - np.count_nonzero(arr_walkable) / (arr_walkable.shape[0]*arr_walkable.shape[1])

        ## Shortest path
        if not self.shortest_path is None:
            logger.info(f"Shortest path already computed, shortest_path={self.shortest_path}. Continue.")
        try:
            shortest_path = self._solve_shortestpath_in_dungeon()
            if shortest_path is None:
                logging.error(f"No shortest path found")
                return None
            self.shortest_path = shortest_path

        except ValueError as ve:
            logging.error(f"Dungeon shortest path failed, with {ve}")

        feature_dict["spath_length"] = len(self.shortest_path)

        ## Rooms in Shortest Path
        spath_rooms=[]
        for pt in self.shortest_path:
            spath_rooms.append(self.dungeonmap[pt[0],pt[1]])
        spath_rooms_types_uniques=set(spath_rooms)
        # spath_rooms_sequence_of_rooms = [k for k, g in itertools.groupby(spath_rooms)]
        spath_rooms_sequence_of_rooms_corridorrless = [k for k, g in itertools.groupby(spath_rooms) if k != 1]
        # logger.debug(f"spath_rooms_sequence_of_rooms={spath_rooms_sequence_of_rooms}, corridorless={spath_rooms_sequence_of_rooms_corridorrless}")

        ### Completeness, the ratio of the types of rooms crossed by the shortest path, on the number of possible rooms let by the Genotype. If =1, all the Genotype' room types are crossed
        feature_dict["spath_completeness"] = len(spath_rooms_types_uniques) / 6       # With 6 the number of rooms types (entrance, corridor, monster, exit...) 

        ### Diversity, ratio that measures the variety of rooms crossed with a score computed on the good alternance of rooms types (not always the same)
        score=0
        for i in range(len(spath_rooms_sequence_of_rooms_corridorrless)-1):
            if spath_rooms_sequence_of_rooms_corridorrless[i] != spath_rooms_sequence_of_rooms_corridorrless[i+1]:
                score+=1
        if (len(spath_rooms_sequence_of_rooms_corridorrless)-1) <= 0:
            feature_dict["spath_diversity"] = 0
        else:
            feature_dict["spath_diversity"] = score/(len(spath_rooms_sequence_of_rooms_corridorrless)-1)

        ### spath_corridorness, 
        spath_corridors_count = spath_rooms.count(1)
        # print(f"nb_corridors={spath_rooms.count(1)} nb_others={len(spath_rooms)-spath_rooms.count(1)}")
        feature_dict["spath_corridorness"] = spath_corridors_count / len(spath_rooms)
        
        return feature_dict

    def _get_walkable_dungeon_surfaces(self) -> np.ndarray:
        if not isinstance(self.dungeonmap, np.ndarray):
            raise ValueError(f"No dungeon map has been provided, dungeonmap={self.dungeonmap}")
        convert_to_ones_and_zeros = np.vectorize(lambda x: 1 if x == 0 else 0, otypes=[int])
        return convert_to_ones_and_zeros(self.dungeonmap)
    
    def _solve_shortestpath_in_dungeon(self): # -> np.ndarray
        arr_walkable = self._get_walkable_dungeon_surfaces()
        walkable_dungeon_as_maze = AStarMaze(maze=arr_walkable)
        d_in, d_out = self.get_dungeon_entrypoints_positions(upscaled=True)
        shortest_path=walkable_dungeon_as_maze.solve_shortest_path(start=d_in, end=d_out)
        return shortest_path


    # Generate
    def grow_dungeonmap_from_genotype(self):
        logger.info(f"--- Growing dungeonmap with {len(self.genotype.pois)} pois ---")
        if not isinstance(self.genotype, Genotype):
            raise ValueError(f"No valid Genotype found, can't grow a dungeon from {type(self.genotype)}")

        # Initialize the dungeonmap, as a np array with the phenotypal POIs
        arr_phenotypal = np.zeros(self.dungeon_shape)
        for poi in self.genotype.pois:
            arr_phenotypal[poi[0]][poi[1]] = poi[2]

        # Draw the dungeon rooms by growing each POI'region
        arr_grownrooms = self._get_dungeonrooms_by_growing_regions(arr_dungeonmap=arr_phenotypal, iteration_limit=self.genotype.growth_iterator_limit)
        # self._preview_dungeon(arr=arr_grownrooms)

        # Draw the dungeon paths, from the POI minimum spanning tree
        G_delaunay, T_minspan = self._build_utils_graphs_from_genotype()
        g_circu = self._build_circulation_graph(G_delaunay=G_delaunay, T_minspan=T_minspan, add_edges=1)
        logger.debug(f"Dungeon graphs edges: delaunay={len(G_delaunay.edges)}, minimum span tree={len(T_minspan.edges)}, circulation={len(g_circu.edges)}")
        arr_dungeonpaths = self._get_dungeonpath_from_circulation_graph(G_circulation=g_circu, dungeon_shape=self.dungeon_shape, pois=self.genotype.pois)
        self.dungeonmap = self._get_upscaled_dungeon_with_rooms_and_paths(arr_rooms=arr_grownrooms, G_circulation=g_circu)
        logger.debug(f"Upscaled to shape {arr_grownrooms.shape}")

    def _get_upscaled_dungeon_with_rooms_and_paths(self, arr_rooms, G_circulation):
        def upscale_dungeonroom_array(arr_rooms):
            upscaled = np.full(np.dot(arr_rooms.shape, 2), -1)
            for x in range(arr_rooms.shape[0]):
                for y in range(arr_rooms.shape[1]):
                    upscaled[x*2][y*2] = arr_rooms[x][y]
            return upscaled
        
        up_base  = upscale_dungeonroom_array(arr_rooms=arr_rooms)
        up_final = np.copy(up_base)
        up_pois  = np.hstack([np.dot(self.genotype.pois[:,:2],2), self.genotype.pois[:,2:]])
        
        up_paths = self._get_dungeonpath_from_circulation_graph(dungeon_shape=up_base.shape, G_circulation=G_circulation, pois=up_pois)

        for x in range(0, up_base.shape[0], 2):
            for y in range(0, up_base.shape[1]-1):
                if up_final[x][y] == -1:
                    # if up_paths[x][y] == 1:
                    #     up_final[x][y] = 10
                    if up_final[x][y-1] == up_final[x][y+1]:
                        # print(f'equal {up_base_pathed[x][y-1]}')
                        up_final[x][y] = up_final[x][y-1]

        for x in range(0, up_base.shape[0]-1):
            for y in range(up_base.shape[1]):
                if up_final[x][y] == -1:
                    # if up_paths[x][y] == 1:
                    #     up_final[x][y] = 10
                    if up_final[x-1][y] == up_final[x+1][y]:
                        # print(f'equal {up_base_pathed[x][y-1]}')
                        up_final[x][y] = up_final[x-1][y]  
        up_rooms_and_paths=np.maximum(up_final,up_paths)
        return up_rooms_and_paths

    ## Utils, rooms
    def _get_dungeonrooms_by_growing_regions(self, arr_dungeonmap: np.ndarray, iteration_limit=100) -> np.ndarray:
        def grow_regions(arr_dungeonmap: np.ndarray):
            if not len(arr_dungeonmap.shape) == 2:
                raise ValueError(f"Array map shall be flat 2 dimensional, actual shape is {arr_dungeonmap.shape}")

            nplus_map = np.zeros(arr_dungeonmap.shape)
            adj_coords = {(-1, 1), (0, 1), (1, 1), (-1, 0), (1, 0), (-1, -1), (0,-1), (1,-1)}

            for x in range(arr_dungeonmap.shape[0]):
                for y in range(arr_dungeonmap.shape[1]):
                    if arr_dungeonmap[x][y] > 0:
                        nplus_map[x][y] = arr_dungeonmap[x][y]
                        for dx, dy in adj_coords:
                            if 0 <= x+dx < arr_dungeonmap.shape[0] and 0 <= y+dy < arr_dungeonmap.shape[1]:
                                if arr_dungeonmap[x+dx][y+dy] == 0:
                                    nplus_map[x+dx][y+dy] = arr_dungeonmap[x][y]
            return nplus_map
        
        count = 0
        grown_map = np.copy(arr_dungeonmap)
        while 0 in grown_map and count < iteration_limit:
            logger.info(f"Iteration {count}, actual empty spaces={np.count_nonzero(grown_map == 0)}")
            logging.debug(f"Growing pois regions, iteration={count}")
            grown_map = grow_regions(arr_dungeonmap=grown_map)
            count+=1
        return grown_map

    def _get_phenotypal_dungeon(self) -> np.ndarray:
        if not isinstance(self.genotype, Genotype):
            raise ValueError(f"No valid Genotype found, can't grow a dungeon from {type(self.genotype)}")      
        arr = np.zeros(self.dungeon_shape)
        for poi in self.genotype.pois:
            arr[poi[0]][poi[1]] = poi[2]
        return arr
    
    ## Utils, paths
    def _get_dungeonpath_from_circulation_graph(self, dungeon_shape, G_circulation, pois):
        arr_dungeonpath = np.zeros(dungeon_shape)
        for e in G_circulation.edges:
            p0 = pois[e[0]][:2]
            p1 = pois[e[1]][:2]
            new_line = self._line_get_angle(p0[0], p0[1], p1[0], p1[1], False)
            logging.debug(e, new_line)
            for pt in new_line:
                arr_dungeonpath[pt[0]][pt[1]] = 1
        return arr_dungeonpath

    def _build_utils_graphs_from_genotype(self) -> Tuple[nx.Graph, nx.Graph]:
        """
        Given an array-like of 2D points, return a NetworkX graph of the Delaunay triangulation,
        with edges weighted by cityblock (Manhattan) distance.
        """
        pois = np.asarray(self.genotype.pois, dtype=int)
        pos = pois[:, :2]  # only x, y
        tri = Delaunay(pos)

        # Extract edges from Delaunay triangles
        edges = set()
        for simplex in tri.simplices:
            for i, j in itertools.combinations(simplex, 2):
                edges.add(tuple(sorted((int(i), int(j)))))

        # Compute weights using cityblock distance
        e1, e2 = zip(*edges)
        weights = cdist(pos[list(e1)], pos[list(e2)], metric='cityblock')
        weighted_edges = [(i, j, w) for (i, j), w in zip(edges, weights.diagonal())]

        # Build graph
        G_delaunay = nx.Graph()
        G_delaunay.add_weighted_edges_from(weighted_edges)
        T_minspan=nx.minimum_spanning_tree(G_delaunay)
        return G_delaunay, T_minspan

    def _build_circulation_graph(self, G_delaunay, T_minspan, add_edges=0):
        G = deepcopy(T_minspan)
        g_diff = nx.difference(G_delaunay, T_minspan)
        if add_edges > len(g_diff.edges):
            logging.debug(f"add_edges={add_edges}, the Delaunay Graph is returned")
            return G_delaunay
        
        listof_new_edges = list(g_diff.edges)
        logging.debug(f"adding edges {listof_new_edges[:add_edges]} to the minimum spanning tree")
        G.add_edges_from(listof_new_edges[:add_edges])
        return G

    def _debug_plot_graph(self, G, pos, node_size=30, font_size=8):
        """
        Visualizes the graph with edge weights and labels.
        """
        plt.figure(figsize=(3, 3))
        nx.draw_networkx_edges(G, pos, width=2)
        nx.draw_networkx_labels(G, pos, font_size=font_size)
        edge_labels = nx.get_edge_attributes(G, "weight")
        nx.draw_networkx_edge_labels(G, pos, edge_labels, font_size=font_size)
        nx.draw_networkx_nodes(G, pos, node_size=node_size)

        plt.axis("off")
        plt.tight_layout()
        plt.show()

    ### Line Drawing
    def _line_get_bresenham(self, x0,y0,x1,y1):
        points_line = []
        dx = abs(x1 - x0)
        sx = 1 if x0 < x1 else -1
        dy = -abs(y1 - y0)
        sy = 1 if y0 < y1 else -1
        error = dx + dy
        while True:
            logging.debug(x0, y0)
            points_line.append((x0, y0))
            e2 = 2 * error
            if e2 >= dy:
                if x0 == x1:
                    break
                error = error + dy
                x0 = x0 + sx
            if e2 <= dx:
                if y0 == y1:
                    break
                error = error + dx
                y0 = y0 + sy
        return points_line

    def _line_get_angle(self, x0, y0, x1, y1, l_shaped_angle=True):
        if x0 == x1:  # vertical
            return [(x0, y) for y in range(min(y0, y1), max(y0, y1)+1)]
        
        if y0 == y1:  # horizontal
            return [(x, y0) for x in range(min(x0, x1), max(x0, x1)+1)]
        
        if l_shaped_angle:
            # First vertical, then horizontal
            vline = [(x0, y) for y in range(min(y0, y1), max(y0, y1)+1)]
            hline = [(x, y1) for x in range(min(x0, x1), max(x0, x1)+1)]
            logging.debug(f"L-shape: vline={vline}, hline={hline}")
            return vline + vline[1:]
        else:
            # First horizontal, then vertical
            hline = [(x, y0) for x in range(min(x0, x1), max(x0, x1)+1)]
            vline = [(x1, y) for y in range(min(y0, y1), max(y0, y1)+1)]
            logging.debug(f"reverse-L-shape: hline={hline}, vline={vline}")
            return hline + vline[1:]

    def _debug_preview_line_draw(self, line_get_function):
        endpoints=[(1,1),(1,6),(6,1),(6,8)]
        for e in itertools.islice(itertools.combinations(endpoints, 2),6):
            print(e)
            points_in_line = line_get_function(e[0][0],e[0][1],e[1][0],e[1][1], False)
            preview_line_arr=np.zeros((10,10))
            for p in points_in_line:
                preview_line_arr[p[0]][p[1]] = 1
            self._preview_dungeon(preview_line_arr)

    # Previews
    def _preview_dungeon(self, arr: np.ndarray, palette=None):
        """
        Displays a 2D numpy array using a custom palette for values from 0 to 10.
        """
        if palette is None:
            # Default: tab10 colors (10 colors max)
            palette = [
                "#ffffff", "#b4b4b4", "#8ad98a", "#145221", "#c9c9c9",
                "#c13e3e", "#dbbd24", "#973636", "#bcbd22", "#17becf", "#000000"
            ]

        cmap = ListedColormap(palette[:11])  # Use only the first 11 colors

        fig, ax = plt.subplots()
        cax = ax.imshow(arr, vmin=0, vmax=10, cmap=cmap, interpolation='nearest', aspect='equal')

        # Colorbar with discrete ticks
        # cbar = plt.colorbar(cax, ticks=range(11))
        # cbar.set_label("Tile Type")

        # Ticks & layout
        ax.set_xticks(np.arange(arr.shape[1]))
        ax.set_yticks(np.arange(arr.shape[0]))
        ax.invert_yaxis()
        # ax.set_title("Dungeon Preview")

        plt.tight_layout()
        plt.show()
        
    def preview_full_dungeon(self) -> None:
        if not isinstance(self.dungeonmap, np.ndarray):
            raise ValueError(f"No dungeon map has been provided, dungeonmap={self.dungeonmap}")
        self._preview_dungeon(self.dungeonmap)
    
    def preview_phenotypal_dungeon(self) -> None:
        arr=self._get_phenotypal_dungeon()
        self._preview_dungeon(arr)
    
    def preview_walkable_dungeon(self) -> None:
        walkable_palette = [
                "#ffffff", "#000000", "#8ad98a", "#145221", "#c9c9c9",
                "#c13e3e", "#dbbd24", "#973636", "#bcbd22", "#17becf", "#000000"
            ]
        arr=self._get_walkable_dungeon_surfaces()
        self._preview_dungeon(arr, palette=walkable_palette)

    def preview_speedrunnable_dungeon(self) -> None:
        walkable_palette = [
                "#ffffff", "#000000", "#8ad98a", "#145221", "#c9c9c9",
                "#c13e3e", "#dbbd24", "#973636", "#bcbd22", "#17becf", "#000000"
            ]
        arr=self._get_walkable_dungeon_surfaces()

        if self.shortest_path is None:
            raise ValueError(f"Compute the dungeon features first")
        for pt in self.shortest_path:
            arr[pt[0],pt[1]] = 4
        d_in, d_out = self.get_dungeon_entrypoints_positions(upscaled=True)
        arr[d_in[0],d_in[1]] = 2
        arr[d_out[0],d_out[1]] = 3
        self._preview_dungeon(arr, palette=walkable_palette)

### Walk that dungeon

In [ ]:
dim=10
genotype = Genotype()
genotype.generate_genotype(
    dungeon_shape=(dim,dim), 
    number_of_pois=6,
    growth_iterator_limit=2
    )
logger.debug(genotype)

In [ ]:
dmap = DungeonMap(dimension=dim, genotype=genotype)

dmap.grow_dungeonmap_from_genotype()
# dmap.preview_phenotypal_dungeon()
# dmap.preview_full_dungeon()
# dmap.preview_walkable_dungeon()

display(dmap.extract_dungeon_features())
dmap.preview_speedrunnable_dungeon()

In [ ]:
arr_walkable = dmap._get_walkable_dungeon_surfaces()
walkable_dungeon_as_maze = AStarMaze(maze=arr_walkable)

d_in, d_out = dmap.get_dungeon_entrypoints_positions(upscaled=True)

# shortest_path=walkable_dungeon_as_maze.solve_shortest_path(start=d_in, end=d_out)
# shortest_path
arr = dmap._get_walkable_dungeon_surfaces()
arr[d_in[0],d_in[1]] = 2
arr[d_out[0],d_out[1]] = 3
dmap._preview_dungeon(arr, palette = [
                "#ffffff", "#000000", "#8ad98a", "#145221", "#c9c9c9",
                "#c13e3e", "#dbbd24", "#973636", "#bcbd22", "#17becf", "#000000"
            ])

In [ ]:
dmap.extract_dungeon_features()

# Drafting

## Indivual `IndividualDungeon` and Mutations

In [ ]:
class DungeonIndividual(Genotype):
    def __init__(self):
        super().__init__()
        self._id = str(uuid.uuid4())[:8]
    
    def get_dungeon_features_as_phenotype(self, preview=False):
        logger.debug(f"Genotype: {self}")
        if self.pois is None:
            raise ValueError(f"Initialize the individual genotype first to setup the pois")
        
        dmap = DungeonMap(dimension=self.dungeon_shape[0], genotype=self)
        dmap.grow_dungeonmap_from_genotype()
        if preview is True:
            dmap.preview_full_dungeon()
        dungeon_features = dmap.extract_dungeon_features()
        self.dungeon_features_as_phenotype = dungeon_features
        return True

In [ ]:
di = DungeonIndividual()
di.generate_genotype(dungeon_shape=(16,16), number_of_pois=5, growth_iterator_limit=2)
di.get_dungeon_features_as_phenotype(preview=True)
# di.dungeon_features_as_phenotype
display(di)

In [ ]:
# for i in range(3):
#     dib = DungeonIndividual()
#     dib.mutate_from_genotype(genotomut=di)
#     display(dib)
#     dib.get_dungeon_features_as_phenotype(preview=True)

### Populations of the `DungeonEvolutionLord`

In [ ]:
import copy

class DungeonEvolutionLord():
    def __init__(self, tot_pop:int = 10) -> None:
        self.dungeon_fixed_genotype_shape=(16,16)    # Shape of the genotype is half of the final dungeon. Prefer pow² values
        self.mu_pop = []
        self.lam_pop = []
        
        self.popcount       = tot_pop
        self.popcount_mu    = round( (1/6) * tot_pop ) # because 1/8 it's "a good rule of thumb" https://algorithmafternoon.com/strategies/mu_plus_lambda_evolution_strategy/
        self.popcount_lam   = tot_pop - self.popcount_mu

    # Population
    def initialize_mu_population(self):
        logger.info(f"--- Initializing Mu Population with {self.popcount_mu} individuals")
        for i in range(self.popcount_mu):
            di = DungeonIndividual()
            di.generate_genotype(dungeon_shape=self.dungeon_fixed_genotype_shape, number_of_pois=5, growth_iterator_limit=2)
            di.get_dungeon_features_as_phenotype(preview=False)
            self.mu_pop.append(di)
    
    def generate_lam_population(self, global_mutation_strength:float):
        for i in range(self.popcount_lam):
            parent_mu_individual = random.sample(self.mu_pop, k=1)[0]
            mutated_lam_individuel = DungeonIndividual()
            mutated_lam_individuel.mutate_from_genotype(genotomut=parent_mu_individual)
            mutated_lam_individuel.get_dungeon_features_as_phenotype(preview=False)
            self.lam_pop.append(mutated_lam_individuel)


    def _reset_pops(self):
        self.mu_pop = []
        self.lam_pop = []
    
    def _evaluate_tot_population(self, fitness_function):
        df_pop = self.get_population_phenotypes_dataframe(pop_name='all')
        df_pop[['fit_slength', 'fit_corridorness', 'fit_value']] = df_pop.apply(fitness_function, axis=1)
        return df_pop


    def evolve_dungeon(self, evolve_iterations=11):
        logger.warning(f"--- Evolving for {evolve_iterations} generations ---")
        cube_generation = np.ndarray((self.popcount,10, evolve_iterations), dtype=object)
        global_mutation_strength = 1.0
        mutation_decrease_treshold = 0.72

        fitness_column_name= 'fit_slength'

        for i in range(evolve_iterations):
            logger.warning(f"---- START Generation {i} ----")
            self.generate_lam_population(global_mutation_strength=global_mutation_strength)

            df_pop = self._evaluate_tot_population(self._fitness_A).sort_values(by=[fitness_column_name], ascending=False)
            fitness_description = df_pop[fitness_column_name].describe()[['count', 'mean', 'std', 'min', 'max']].round(2).to_list()
            fitness_value_top_individuals = df_pop[:self.popcount_mu][fitness_column_name]
            logger.warning("Gen {} fitness: {}pop, mean={}, std={}, min={}, max={} ; tops_to_mu={}".format(*([i] + fitness_description + [fitness_value_top_individuals.round(3).tolist()])))

            # Saving to the cube
            cube_generation[:,:,i] = df_pop.to_numpy()

            if i < evolve_iterations - 1:
                number_of_individuals_performing_up_trehshold = len(df_pop[df_pop[fitness_column_name] > mutation_decrease_treshold])
                if ( number_of_individuals_performing_up_trehshold > 0.2 * self.popcount) and global_mutation_strength > 0.0:
                    logger.warning(f"{number_of_individuals_performing_up_trehshold} performs better than {mutation_decrease_treshold}, decreasing mutation rate {global_mutation_strength}->{global_mutation_strength-0.2}  ({0.2 * self.popcount})")
                    global_mutation_strength -= 0.2
                else:
                    logger.warning(f"{number_of_individuals_performing_up_trehshold} performs better than {mutation_decrease_treshold}, keeping mutation rate {global_mutation_strength} (trsh{0.2 * self.popcount})")

                new_mu_population = copy.deepcopy([self.get_individual_in_actual_pops(indi_id) for indi_id in df_pop[:self.popcount_mu].index.to_list()])
                logger.warning(f"Pops Reset")
                self._reset_pops()
                self.mu_pop = new_mu_population
                logger.warning(f"Ending iteration with selection of mu population={new_mu_population}")
                logger.warning(f"---- END {i} ----")
        
        logger.warning(f"---- END Evolving ----")
        return cube_generation

    # Getters, for population
    def get_population_phenotypes_dataframe(self, pop_name:str = 'mu'):
        if pop_name == 'mu':
            population = self.mu_pop
        elif pop_name == 'lam':
            population = self.lam_pop
        elif pop_name == 'all':
            return pd.concat([self.get_population_phenotypes_dataframe(pop_name='mu'), self.get_population_phenotypes_dataframe(pop_name='lam')])

        else:
            raise ValueError(f"pop_name must be 'mu', 'lam' or 'all': not {pop_name}")

        data = []
        for ind in population:
            phenotype = ind.dungeon_features_as_phenotype
            phenotype['_id'] = ind._id          # Add ID to the row
            phenotype['pop'] = pop_name         # Add population identifier
            data.append(phenotype)
            
        df = pd.DataFrame(data).set_index('_id')
        return df

    def get_population_genotype_dataframe(self, pop_name:str = 'mu'):
        pass


    def get_individual_in_actual_pops(self, id):
        for indimu in self.mu_pop:
            if indimu._id == id:
                return indimu
            
        for indilam in self.lam_pop:
            if indilam._id == id:
                return indilam
        return None

    # Fitnesses
    def _fitness_A(self, row):
        """ Fitness function for evalutating a Dungeon Individual, based on its phenotype.
            Evaluation based on the length of the shortest path between entrance and exit, and the fraction of corridors in this path
        Args:
            row (_type_): _description_

        Returns:
            _type_: _description_
        """
        goal_spath_length = 18 # self.dungeon_fixed_genotype_shape[0] * 0.2
        # fitnessval_spath_length = 1 / (1 + abs(row['spath_length'] - goal_spath_length)) # inverse linear
        fitnessval_spath_length = 1 / (1 + pow(row['spath_length'] - goal_spath_length, 2)) # inverse squared
        goal_spath_corridorness = 0.2
        # fitnessval_spath_corridorness = 1 - abs(row['spath_corridorness'] - goal_spath_corridorness) # inverse abs distance
        fitnessval_spath_corridorness = 1 / (1 + abs(row['spath_corridorness'] - goal_spath_corridorness))
        fitness_weighted_sum_of_distances = 0.25 * fitnessval_spath_length + 0.75 * fitnessval_spath_corridorness
        return pd.Series([fitnessval_spath_length, fitnessval_spath_corridorness, fitness_weighted_sum_of_distances], 
                        index=['fitnessval_slength', 'fitnessval_corridorness', 'fitness_weighted_sum_of_distances'])

In [ ]:
delord = DungeonEvolutionLord(tot_pop=20)
delord.initialize_mu_population()
# delord.get_population_phenotypes_dataframe()
# delord.generate_lam_population(global_mutation_strength=1.0)

In [ ]:
delord = DungeonEvolutionLord(tot_pop=40)
delord.initialize_mu_population()
cube_gens = delord.evolve_dungeon(evolve_iterations=8)

In [ ]:
# delord.get_population_phenotypes_dataframe(pop_name='all')
delord.lam_pop
df = pd.DataFrame(data=cube_gens[:,:,0], columns=['corridorness', 'nolliness', 'spath_length', 'spath_completeness', 'spath_diversity', 'spath_corridorness', 'pop', "fit1", "fit2", "fit3"])
df

In [ ]:
delord._evaluate_tot_population(fitness_function=delord._fitness_A).sort_values(by=['fit_slength'], ascending=False)

In [ ]:
data = cube_gens
generations=8
total_population=40

col_index_spath = 4
col_index_fit   = 7

# --- Plotting ---
fig, axs = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Box plot of Fitness (feature col_index_fit) over all generations
fitness_over_time = data[:, 2, :]  # shape: (individuals, generations)
box_data = [fitness_over_time[:, gen] for gen in range(generations)]

axs[0].boxplot(box_data, positions=np.arange(generations), patch_artist=True)
axs[0].set_title('Fitness Over Generations')
axs[0].set_xlabel('Generation')
axs[0].set_ylabel('Fitness')
axs[0].set_xlim(-1, generations)
axs[0].grid(True)

# Plot 2: Spath_length (feature 4) over generations with opacity based on final fitness
spath_length = data[:, col_index_fit, :]
fitness_last_gen = fitness_over_time[:, -1]
fitness_ranking = fitness_last_gen.argsort()
alphas = np.linspace(0.1, 1.0, total_population)[fitness_ranking.argsort()]

for i in range(total_population):
    axs[1].plot(spath_length[i], color='blue', alpha=alphas[i])

axs[1].set_title('Spath Length Over Generations')
axs[1].set_xlabel('Generation')
axs[1].set_ylabel('Spath Length')
axs[1].grid(True)

plt.tight_layout()
plt.show()

## `Genotype` and Genotype Expansion

In [ ]:
def initialize_dungeonmap(dimension: int):
    def get_random_pois(array_shape: tuple, n_pois: int, n_typeof_pois: int):
        """Generate a set of localized Points of Interest in a dungeon map represented as an array.

        Args:
            dimension (int): _description_
            n_pois (int): _description_
            n_typeof_pois (int): _description_

        Raises:
            ValueError: _description_

        Returns:
            _type_: _description_
        """
        if n_pois < 3:
            raise ValueError("number of pois shall be > 3, we Delaunay triangulate pois later")
        return [[randrange(array_shape[0]), randrange(array_shape[1]), randrange(n_typeof_pois)+2] for n in range(n_pois)]

    arr = np.zeros((dimension,dimension))
    random_pois = get_random_pois(array_shape=arr.shape, n_pois=5, n_typeof_pois=5)

    for poi in random_pois:
        arr[poi[0]][poi[1]] = poi[2]
    return arr, random_pois

def preview_dungeon(arr: np.ndarray, palette=None):
    """
    Displays a 2D numpy array using a custom palette for values from 0 to 10.
    """
    if palette is None:
        # Default: tab10 colors (10 colors max)
        palette = [
            "#ffffff", "#b4b4b4", "#2ca02c", "#d62728", "#9467bd",
            "#8c564b", "#e377c2", "#973636", "#bcbd22", "#17becf", "#000000"
        ]

    cmap = ListedColormap(palette[:11])  # Use only the first 11 colors

    fig, ax = plt.subplots()
    cax = ax.imshow(arr, vmin=0, vmax=10, cmap=cmap, interpolation='nearest', aspect='equal')

    # Colorbar with discrete ticks
    # cbar = plt.colorbar(cax, ticks=range(11))
    # cbar.set_label("Tile Type")

    # Ticks & layout
    ax.set_xticks(np.arange(arr.shape[1]))
    ax.set_yticks(np.arange(arr.shape[0]))
    ax.invert_yaxis()
    # ax.set_title("Dungeon Preview")

    plt.tight_layout()
    plt.show()


dim = 7
arr_d, pois = initialize_dungeonmap(dimension=dim)
np_pois = np.asarray(pois, dtype=int)
preview_dungeon(arr=arr_d)

### Dungeon Rooms

In [ ]:
def draw_dungeonrooms_by_growing_regions(arr_dungeonmap: np.ndarray, iteration_limit=100):
    def grow_regions(arr_dungeonmap: np.ndarray):
        if not len(arr_dungeonmap.shape) == 2:
            logging.error(f"Array map shall be flat 2 dimensional, actual shape is {arr_dungeonmap.shape}")

        nplus_map = np.zeros(arr_dungeonmap.shape)
        adj_coords = {(-1, 1), (0, 1), (1, 1), (-1, 0), (1, 0), (-1, -1), (0,-1), (1,-1)}

        for x in range(arr_dungeonmap.shape[0]):
            for y in range(arr_dungeonmap.shape[1]):
                if arr_dungeonmap[x][y] > 0:
                    nplus_map[x][y] = arr_dungeonmap[x][y]
                    for dx, dy in adj_coords:
                        if 0 <= x+dx < arr_dungeonmap.shape[0] and 0 <= y+dy < arr_dungeonmap.shape[1]:
                            if arr_dungeonmap[x+dx][y+dy] == 0:
                                nplus_map[x+dx][y+dy] = arr_dungeonmap[x][y]
        return nplus_map
    
    count = 0
    grown_map = np.copy(arr_dungeonmap)
    while 0 in grown_map and count <= iteration_limit:
        logging.debug(f"Growing pois regions, iteration={count}")
        grown_map = grow_regions(arr_dungeonmap=grown_map)
        count+=1
    return grown_map

In [ ]:
arr_g=draw_dungeonrooms_by_growing_regions(arr_dungeonmap=arr_d, iteration_limit=1)
preview_dungeon(arr=arr_g)

#### Circulation Graph

In [ ]:
def build_graphs_delaunay_minspan(pois):
    """
    Given an array-like of 2D points, return a NetworkX graph of the Delaunay triangulation,
    with edges weighted by cityblock (Manhattan) distance.
    """
    pois = np.asarray(pois, dtype=int)
    pos = pois[:, :2]  # only x, y
    tri = Delaunay(pos)

    # Extract edges from Delaunay triangles
    edges = set()
    for simplex in tri.simplices:
        for i, j in itertools.combinations(simplex, 2):
            edges.add(tuple(sorted((int(i), int(j)))))

    # Compute weights using cityblock distance
    e1, e2 = zip(*edges)
    weights = cdist(pos[list(e1)], pos[list(e2)], metric='cityblock')
    weighted_edges = [(i, j, w) for (i, j), w in zip(edges, weights.diagonal())]

    # Build graph
    G = nx.Graph()
    G.add_weighted_edges_from(weighted_edges)
    T=nx.minimum_spanning_tree(G)
    return G, T

def build_circulation_graph(G_delaunay, T_minspan, add_edges=0):
    G = deepcopy(T_minspan)
    g_diff = nx.difference(G_delaunay, T_minspan)
    if add_edges > len(g_diff.edges):
        logging.debug(f"add_edges={add_edges}, the Delaunay Graph is returned")
        return G_delaunay
    
    listof_new_edges = list(g_diff.edges)
    logging.debug(f"adding edges {listof_new_edges[:add_edges]} to the minimum spanning tree")
    G.add_edges_from(listof_new_edges[:add_edges])
    return G

def plot_graph(G, pos, node_size=30, font_size=8):
    """
    Visualizes the graph with edge weights and labels.
    """
    plt.figure(figsize=(3, 3))
    nx.draw_networkx_edges(G, pos, width=2)
    nx.draw_networkx_labels(G, pos, font_size=font_size)
    edge_labels = nx.get_edge_attributes(G, "weight")
    nx.draw_networkx_edge_labels(G, pos, edge_labels, font_size=font_size)
    nx.draw_networkx_nodes(G, pos, node_size=node_size)

    plt.axis("off")
    plt.tight_layout()
    plt.show()


In [ ]:
G_delau2, T = build_graphs_delaunay_minspan(np_pois)
g_circu = build_circulation_graph(G_delau2, T, 1)
# plot_graph(G_delau2, np_pois[:, :2])
# plot_graph(T, np_pois[:, :2])
# plot_graph(g_circu, np_pois[:, :2])

In [ ]:
def line_get_bresenham(x0,y0,x1,y1):
    points_line = []
    dx = abs(x1 - x0)
    sx = 1 if x0 < x1 else -1
    dy = -abs(y1 - y0)
    sy = 1 if y0 < y1 else -1
    error = dx + dy
    while True:
        logging.debug(x0, y0)
        points_line.append((x0, y0))
        e2 = 2 * error
        if e2 >= dy:
            if x0 == x1:
                break
            error = error + dy
            x0 = x0 + sx
        if e2 <= dx:
            if y0 == y1:
                break
            error = error + dx
            y0 = y0 + sy
    return points_line

def line_get_angle(x0, y0, x1, y1, l_shaped_angle=True):
    if x0 == x1:  # vertical
        return [(x0, y) for y in range(min(y0, y1), max(y0, y1)+1)]
    
    if y0 == y1:  # horizontal
        return [(x, y0) for x in range(min(x0, x1), max(x0, x1)+1)]
    
    if l_shaped_angle:
        # First vertical, then horizontal
        vline = [(x0, y) for y in range(min(y0, y1), max(y0, y1)+1)]
        hline = [(x, y1) for x in range(min(x0, x1), max(x0, x1)+1)]
        logging.debug(f"L-shape: vline={vline}, hline={hline}")
        return vline + vline[1:]
    else:
        # First horizontal, then vertical
        hline = [(x, y0) for x in range(min(x0, x1), max(x0, x1)+1)]
        vline = [(x1, y) for y in range(min(y0, y1), max(y0, y1)+1)]
        logging.debug(f"reverse-L-shape: hline={hline}, vline={vline}")
        return hline + vline[1:]

def _debug_preview_line_draw(line_get_function):
    endpoints=[(1,1),(1,6),(6,1),(6,8)]
    for e in itertools.islice(itertools.combinations(endpoints, 2),6):
        print(e)
        points_in_line = line_get_function(e[0][0],e[0][1],e[1][0],e[1][1], False)
        preview_line_arr=np.zeros((10,10))
        for p in points_in_line:
            preview_line_arr[p[0]][p[1]] = 1
        preview_dungeon(preview_line_arr)
# preview_line_draw(line_get_bresenham)
# _debug_preview_line_draw(line_get_angle)
plot_graph(g_circu, np_pois[:, :2])

In [ ]:
def draw_dungeonpath_from_circulation_graph(dimension, G_circulation, pois):
    arr_dungeonpath = np.zeros((dimension,dimension))
    for e in G_circulation.edges:
        p0 = pois[e[0]][:2]
        p1 = pois[e[1]][:2]
        new_line = line_get_angle(p0[0], p0[1], p1[0], p1[1], False)
        logging.debug(e, new_line)
        for pt in new_line:
            arr_dungeonpath[pt[0]][pt[1]] = 1
    return arr_dungeonpath
arr_p = draw_dungeonpath_from_circulation_graph(dimension=dim, G_circulation=g_circu, pois=pois)

In [ ]:
arr=np.maximum(arr_d,arr_p)
preview_dungeon(arr)

In [ ]:
arr=np.maximum(arr_g,arr_p)
preview_dungeon(arr_g)

### Upscaled

In [ ]:
up_dim=2*dim
up_pois= np.hstack([np.dot(np_pois[:,:2],2), np_pois[:,2:]])

In [ ]:
def draw_upscaled_dungeon_with_rooms_and_paths(arr_dungeonrooms, G_circulation, up_pois):
    def upscale_dungeonroom_array(arr_dungeonrooms):
        upscaled = np.full(np.dot(arr_dungeonrooms.shape, 2), -1)
        for x in range(arr_dungeonrooms.shape[0]):
            for y in range(arr_dungeonrooms.shape[1]):
                upscaled[x*2][y*2] = arr_dungeonrooms[x][y]
        return upscaled
    
    up_base  = upscale_dungeonroom_array(arr_dungeonrooms=arr_dungeonrooms)
    up_final = np.copy(up_base)
    

    up_paths = draw_dungeonpath_from_circulation_graph(dimension=up_base.shape[0], G_circulation=G_circulation, pois=up_pois)

    for x in range(0, up_base.shape[0], 2):
        for y in range(0, up_base.shape[1]-1):
            if up_final[x][y] == -1:
                # if up_paths[x][y] == 1:
                #     up_final[x][y] = 10
                if up_final[x][y-1] == up_final[x][y+1]:
                    # print(f'equal {up_base_pathed[x][y-1]}')
                    up_final[x][y] = up_final[x][y-1]

    for x in range(0, up_base.shape[0]-1):
        for y in range(up_base.shape[1]):
            if up_final[x][y] == -1:
                # if up_paths[x][y] == 1:
                #     up_final[x][y] = 10
                if up_final[x-1][y] == up_final[x+1][y]:
                    # print(f'equal {up_base_pathed[x][y-1]}')
                    up_final[x][y] = up_final[x-1][y]  
    arr=np.maximum(up_final,up_paths)
    return arr

up_dungeon=draw_upscaled_dungeon_with_rooms_and_paths(arr_dungeonrooms=arr_g, G_circulation=g_circu, up_pois=up_pois)
preview_dungeon(up_dungeon)

In [ ]:
arr_g